### CNN Tensorflow Dimensions

Using Convolutional Layers in TensorFlow

Let's now build a convolutional layer in TensorFlow. In the below exercise, you'll be asked to set up the dimensions of the convolution filters, the weights, and the biases. This is in many ways the trickiest part to using CNNs in TensorFlow. Once you have a sense of how to set up the dimensions of these attributes, applying CNNs will be far more straightforward.

You should go over the TensorFlow documentation for [2D convolutions](https://www.tensorflow.org/api_guides/python/nn#Convolution). Most of the documentation is straightforward, except perhaps the padding argument. The padding might differ depending on whether you pass 'VALID' or 'SAME'.

Here are a few more things worth reviewing:

Introduction to TensorFlow -> TensorFlow Variables.
How to determine the dimensions of the output based on the input size and the filter size (shown below). You'll use this to determine what the size of your filter should be.

     new_height = (input_height - filter_height + 2 * P)/S + 1
     new_width = (input_width - filter_width + 2 * P)/S + 1

Instructions

Finish off each TODO in the conv2d function.
Set up the strides, padding, filter weight (F_w), and filter bias (F_b) such that the output shape is (1, 2, 2, 3). Note that all of these except strides should be TensorFlow variables.
    
With padding = 'VALID':

    out_height = ceil(float(in_height - filter_height + 1) / float(strides[1]))
    out_width  = ceil(float(in_width - filter_width + 1) / float(strides[2]))

In [4]:
"""
Setup the strides, padding and filter weight/bias such that
the output shape is (1, 2, 2, 3).
"""
import tensorflow as tf
import numpy as np

# `tf.nn.conv2d` requires the input be 4D (batch_size, height, width, depth)
# (1, 4, 4, 1)
x = np.array([
    [0, 1, 0.5, 10],
    [2, 2.5, 1, -8],
    [4, 0, 5, 6],
    [15, 1, 2, 3]], dtype=np.float32).reshape((1, 4, 4, 1))
X = tf.constant(x)

# convolution filter dimensions
filter_size_width = 2
filter_size_height = 2
input_depth = 1
output_depth = 3

def conv2d(input):
    # Filter (weights and bias)
    # The shape of the filter weight is (height, width, input_depth, output_depth)
    # The shape of the filter bias is (output_depth,)
    # TODO: Define the filter weights `F_W` and filter bias `F_b`.
    # NOTE: Remember to wrap them in `tf.Variable`, they are trainable parameters after all.
    F_W = tf.Variable(tf.truncated_normal([filter_size_height, filter_size_width, input_depth, output_depth]))
    F_b = tf.Variable(tf.zeros(output_depth))
    # TODO: Set the stride for each dimension (batch_size, height, width, depth)
    strides = [1, 2, 2, 1]
    # TODO: set the padding, either 'VALID' or 'SAME'.
    padding = 'VALID'  # with padding VALID since is no padding, can get a smaller output since last stride to last window doesnt happen
    # https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#conv2d
    # `tf.nn.conv2d` does not include the bias computation so we have to add it ourselves after.
    return tf.nn.conv2d(input, F_W, strides, padding) + F_b

out = conv2d(X)
print(out.shape)

(1, 2, 2, 3)


### Max Pooling

Conceptually, the benefit of the max pooling operation is to reduce the size of the input, and allow the neural network to focus on only the most important elements. Max pooling does this by only retaining the maximum value for each filtered area, and removing the remaining values.

TensorFlow provides the tf.nn.max_pool() function to apply max pooling to your convolutional layers.

```
conv_layer = tf.nn.conv2d(input, weight, strides=[1, 2, 2, 1], padding='SAME')
conv_layer = tf.nn.bias_add(conv_layer, bias)
conv_layer = tf.nn.relu(conv_layer)
# apply max pooling
conv_layer = tf.nn.max_pool(
    conv_layer,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding='SAME')
```

The tf.nn.max_pool() function performs max pooling with the ksize parameter as the size of the filter and the strides parameter as the length of the stride. 2x2 filters with a stride of 2x2 are common in practice.

The ksize and strides parameters are structured as 4-element lists, with each element corresponding to a dimension of the input tensor `([batch, height, width, channels])`. For both ksize and strides, the batch and channel dimensions are typically set to 1.

with padding = 'VALID':

```
out_height = ceil(float(in_height - filter_height + 1) / float(strides[1]))
out_width  = ceil(float(in_width - filter_width + 1) / float(strides[2]))
```

In [7]:
"""
Set the values to `strides` and `ksize` such that
the output shape after pooling is (1, 2, 2, 1).
"""
import tensorflow as tf
import numpy as np

# `tf.nn.max_pool` requires the input be 4D (batch_size, height, width, depth)
# (1, 4, 4, 1)
x = np.array([
    [0, 1, 0.5, 10],
    [2, 2.5, 1, -8],
    [4, 0, 5, 6],
    [15, 1, 2, 3]], dtype=np.float32).reshape((1, 4, 4, 1))
X = tf.constant(x)

def maxpool(input):
    # TODO: Set the ksize (filter size) for each dimension (batch_size, height, width, depth)
    ksize = [1, 2, 2, 1]
    # TODO: Set the stride for each dimension (batch_size, height, width, depth)
    strides = [1, 2, 2, 1]
    # TODO: set the padding, either 'VALID' or 'SAME'.
    padding = 'SAME'
    # https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#max_pool
    return tf.nn.max_pool(input, ksize, strides, padding)
    
out = maxpool(X)
print(out.shape)

(1, 2, 2, 1)


### Full CNN Walkthrough

In [28]:
# import the MNIST dataset and using a convenient TensorFlow function to
# batch, scale, and one-hot encode the data

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [29]:
# Store layers weight & bias
# weights: (filter_height, filter_width, input_depth, output_depth)
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

In [30]:
# The conv2d function computes the convolution against weight W,
# and then adds bias b, before applying a ReLU activation function.
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [31]:
# The maxpool2d function applies max pooling to layer x
# using a filter of size k.
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [32]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 28*28*32 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 14*14*64 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    print('conv2.shape', conv2.shape) #conv2.shape (?, 7, 7, 64)
    
    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    print('fc1.shape A', fc1.shape) #fc1.shape A (?, 3136)
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    print('fc1.shape B', fc1.shape) #fc1.shape B (?, 1024)
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [33]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} - Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))


conv2.shape (?, 7, 7, 64)
fc1.shape A (?, 3136)
fc1.shape B (?, 1024)
Epoch  1, Batch   1 - Loss: 63504.8125 Validation Accuracy: 0.136719
Epoch  1, Batch   2 - Loss: 60110.0078 Validation Accuracy: 0.113281
Epoch  1, Batch   3 - Loss: 50694.8750 Validation Accuracy: 0.156250
Epoch  1, Batch   4 - Loss: 45611.5078 Validation Accuracy: 0.136719
Epoch  1, Batch   5 - Loss: 35521.7578 Validation Accuracy: 0.152344
Epoch  1, Batch   6 - Loss: 34439.5703 Validation Accuracy: 0.160156
Epoch  1, Batch   7 - Loss: 36070.6406 Validation Accuracy: 0.160156
Epoch  1, Batch   8 - Loss: 32921.6172 Validation Accuracy: 0.160156
Epoch  1, Batch   9 - Loss: 30002.1367 Validation Accuracy: 0.167969
Epoch  1, Batch  10 - Loss: 31986.3535 Validation Accuracy: 0.152344
Epoch  1, Batch  11 - Loss: 26989.9980 Validation Accuracy: 0.183594
Epoch  1, Batch  12 - Loss: 28523.7266 Validation Accuracy: 0.183594
Epoch  1, Batch  13 - Loss: 24403.8691 Validation Accuracy: 0.179688
Epoch  1, Batch  14 - Loss: 22078

Epoch  1, Batch 119 - Loss:  4419.4810 Validation Accuracy: 0.566406
Epoch  1, Batch 120 - Loss:  5506.8354 Validation Accuracy: 0.566406
Epoch  1, Batch 121 - Loss:  4449.7676 Validation Accuracy: 0.562500
Epoch  1, Batch 122 - Loss:  4430.1973 Validation Accuracy: 0.558594
Epoch  1, Batch 123 - Loss:  4378.0874 Validation Accuracy: 0.582031
Epoch  1, Batch 124 - Loss:  4760.3232 Validation Accuracy: 0.574219
Epoch  1, Batch 125 - Loss:  3951.0391 Validation Accuracy: 0.570312
Epoch  1, Batch 126 - Loss:  5223.9185 Validation Accuracy: 0.570312
Epoch  1, Batch 127 - Loss:  5262.1777 Validation Accuracy: 0.582031
Epoch  1, Batch 128 - Loss:  5601.1846 Validation Accuracy: 0.578125
Epoch  1, Batch 129 - Loss:  4301.5059 Validation Accuracy: 0.582031
Epoch  1, Batch 130 - Loss:  5778.3564 Validation Accuracy: 0.585938
Epoch  1, Batch 131 - Loss:  3824.4749 Validation Accuracy: 0.578125
Epoch  1, Batch 132 - Loss:  5245.5493 Validation Accuracy: 0.578125
Epoch  1, Batch 133 - Loss:  3920.

Epoch  1, Batch 238 - Loss:  3151.3928 Validation Accuracy: 0.652344
Epoch  1, Batch 239 - Loss:  2398.6333 Validation Accuracy: 0.664062
Epoch  1, Batch 240 - Loss:  2229.5647 Validation Accuracy: 0.660156
Epoch  1, Batch 241 - Loss:  2090.4062 Validation Accuracy: 0.667969
Epoch  1, Batch 242 - Loss:  2915.3984 Validation Accuracy: 0.660156
Epoch  1, Batch 243 - Loss:  1763.4460 Validation Accuracy: 0.656250
Epoch  1, Batch 244 - Loss:  2418.6040 Validation Accuracy: 0.660156
Epoch  1, Batch 245 - Loss:  3065.9321 Validation Accuracy: 0.656250
Epoch  1, Batch 246 - Loss:  3406.3979 Validation Accuracy: 0.660156
Epoch  1, Batch 247 - Loss:  3453.4385 Validation Accuracy: 0.671875
Epoch  1, Batch 248 - Loss:  2763.9233 Validation Accuracy: 0.675781
Epoch  1, Batch 249 - Loss:  2480.1155 Validation Accuracy: 0.679688
Epoch  1, Batch 250 - Loss:  3067.3374 Validation Accuracy: 0.667969
Epoch  1, Batch 251 - Loss:  2334.2568 Validation Accuracy: 0.671875
Epoch  1, Batch 252 - Loss:  2468.

Epoch  1, Batch 357 - Loss:  1431.6216 Validation Accuracy: 0.714844
Epoch  1, Batch 358 - Loss:  2122.5337 Validation Accuracy: 0.707031
Epoch  1, Batch 359 - Loss:  2038.6543 Validation Accuracy: 0.695312
Epoch  1, Batch 360 - Loss:  1941.3779 Validation Accuracy: 0.695312
Epoch  1, Batch 361 - Loss:  1516.3711 Validation Accuracy: 0.707031
Epoch  1, Batch 362 - Loss:  2634.3269 Validation Accuracy: 0.699219
Epoch  1, Batch 363 - Loss:  1482.9420 Validation Accuracy: 0.707031
Epoch  1, Batch 364 - Loss:  2663.9072 Validation Accuracy: 0.707031
Epoch  1, Batch 365 - Loss:  3034.2222 Validation Accuracy: 0.707031
Epoch  1, Batch 366 - Loss:  2637.4536 Validation Accuracy: 0.703125
Epoch  1, Batch 367 - Loss:  1874.5361 Validation Accuracy: 0.699219
Epoch  1, Batch 368 - Loss:  2630.0669 Validation Accuracy: 0.703125
Epoch  1, Batch 369 - Loss:  1727.2632 Validation Accuracy: 0.703125
Epoch  1, Batch 370 - Loss:  1742.6892 Validation Accuracy: 0.699219
Epoch  1, Batch 371 - Loss:  2046.

Epoch  2, Batch  47 - Loss:  1639.2820 Validation Accuracy: 0.738281
Epoch  2, Batch  48 - Loss:  1497.0621 Validation Accuracy: 0.738281
Epoch  2, Batch  49 - Loss:  1650.5753 Validation Accuracy: 0.738281
Epoch  2, Batch  50 - Loss:  1691.3997 Validation Accuracy: 0.738281
Epoch  2, Batch  51 - Loss:  1519.2786 Validation Accuracy: 0.726562
Epoch  2, Batch  52 - Loss:  1941.6414 Validation Accuracy: 0.730469
Epoch  2, Batch  53 - Loss:  1287.6765 Validation Accuracy: 0.738281
Epoch  2, Batch  54 - Loss:  1806.5371 Validation Accuracy: 0.738281
Epoch  2, Batch  55 - Loss:  1410.3108 Validation Accuracy: 0.730469
Epoch  2, Batch  56 - Loss:  1561.9172 Validation Accuracy: 0.742188
Epoch  2, Batch  57 - Loss:  1280.5320 Validation Accuracy: 0.742188
Epoch  2, Batch  58 - Loss:  1219.0610 Validation Accuracy: 0.738281
Epoch  2, Batch  59 - Loss:  1541.4604 Validation Accuracy: 0.734375
Epoch  2, Batch  60 - Loss:  1581.6030 Validation Accuracy: 0.734375
Epoch  2, Batch  61 - Loss:  1864.

Epoch  2, Batch 166 - Loss:  1373.4391 Validation Accuracy: 0.750000
Epoch  2, Batch 167 - Loss:   943.9290 Validation Accuracy: 0.742188
Epoch  2, Batch 168 - Loss:  1387.8550 Validation Accuracy: 0.742188
Epoch  2, Batch 169 - Loss:  1255.1333 Validation Accuracy: 0.746094
Epoch  2, Batch 170 - Loss:  1149.0410 Validation Accuracy: 0.738281
Epoch  2, Batch 171 - Loss:  1797.3595 Validation Accuracy: 0.738281
Epoch  2, Batch 172 - Loss:  1352.1077 Validation Accuracy: 0.742188
Epoch  2, Batch 173 - Loss:  2171.3667 Validation Accuracy: 0.742188
Epoch  2, Batch 174 - Loss:  1120.4738 Validation Accuracy: 0.734375
Epoch  2, Batch 175 - Loss:  1321.1785 Validation Accuracy: 0.734375
Epoch  2, Batch 176 - Loss:  1391.4006 Validation Accuracy: 0.738281
Epoch  2, Batch 177 - Loss:  1068.0070 Validation Accuracy: 0.738281
Epoch  2, Batch 178 - Loss:  1523.6455 Validation Accuracy: 0.734375
Epoch  2, Batch 179 - Loss:  1342.7070 Validation Accuracy: 0.734375
Epoch  2, Batch 180 - Loss:  1588.

Epoch  2, Batch 285 - Loss:  1082.4272 Validation Accuracy: 0.742188
Epoch  2, Batch 286 - Loss:   967.2251 Validation Accuracy: 0.734375
Epoch  2, Batch 287 - Loss:  1244.7444 Validation Accuracy: 0.742188
Epoch  2, Batch 288 - Loss:  1112.4598 Validation Accuracy: 0.750000
Epoch  2, Batch 289 - Loss:  1019.6605 Validation Accuracy: 0.746094
Epoch  2, Batch 290 - Loss:   792.2307 Validation Accuracy: 0.761719
Epoch  2, Batch 291 - Loss:  1390.8677 Validation Accuracy: 0.757812
Epoch  2, Batch 292 - Loss:  1160.3370 Validation Accuracy: 0.742188
Epoch  2, Batch 293 - Loss:   897.7365 Validation Accuracy: 0.746094
Epoch  2, Batch 294 - Loss:   994.1781 Validation Accuracy: 0.746094
Epoch  2, Batch 295 - Loss:  1016.6824 Validation Accuracy: 0.746094
Epoch  2, Batch 296 - Loss:  1126.1200 Validation Accuracy: 0.746094
Epoch  2, Batch 297 - Loss:  1465.4984 Validation Accuracy: 0.742188
Epoch  2, Batch 298 - Loss:  1190.1331 Validation Accuracy: 0.746094
Epoch  2, Batch 299 - Loss:  1237.

Epoch  2, Batch 404 - Loss:   846.8173 Validation Accuracy: 0.734375
Epoch  2, Batch 405 - Loss:  1167.6632 Validation Accuracy: 0.738281
Epoch  2, Batch 406 - Loss:  1196.7371 Validation Accuracy: 0.746094
Epoch  2, Batch 407 - Loss:  1256.8811 Validation Accuracy: 0.730469
Epoch  2, Batch 408 - Loss:  1149.9690 Validation Accuracy: 0.738281
Epoch  2, Batch 409 - Loss:  1123.9224 Validation Accuracy: 0.738281
Epoch  2, Batch 410 - Loss:  1656.4128 Validation Accuracy: 0.742188
Epoch  2, Batch 411 - Loss:  1022.2307 Validation Accuracy: 0.738281
Epoch  2, Batch 412 - Loss:   726.0336 Validation Accuracy: 0.730469
Epoch  2, Batch 413 - Loss:  1215.9146 Validation Accuracy: 0.738281
Epoch  2, Batch 414 - Loss:   631.0196 Validation Accuracy: 0.734375
Epoch  2, Batch 415 - Loss:   796.3546 Validation Accuracy: 0.742188
Epoch  2, Batch 416 - Loss:   962.2841 Validation Accuracy: 0.734375
Epoch  2, Batch 417 - Loss:   836.0347 Validation Accuracy: 0.734375
Epoch  2, Batch 418 - Loss:  1273.

Epoch  3, Batch  94 - Loss:  1081.0493 Validation Accuracy: 0.750000
Epoch  3, Batch  95 - Loss:   802.6968 Validation Accuracy: 0.750000
Epoch  3, Batch  96 - Loss:   882.1772 Validation Accuracy: 0.753906
Epoch  3, Batch  97 - Loss:  1073.1251 Validation Accuracy: 0.750000
Epoch  3, Batch  98 - Loss:   607.8214 Validation Accuracy: 0.750000
Epoch  3, Batch  99 - Loss:  1315.8049 Validation Accuracy: 0.746094
Epoch  3, Batch 100 - Loss:   959.0217 Validation Accuracy: 0.742188
Epoch  3, Batch 101 - Loss:  1012.7708 Validation Accuracy: 0.746094
Epoch  3, Batch 102 - Loss:   782.6719 Validation Accuracy: 0.746094
Epoch  3, Batch 103 - Loss:   899.2684 Validation Accuracy: 0.746094
Epoch  3, Batch 104 - Loss:   920.7990 Validation Accuracy: 0.750000
Epoch  3, Batch 105 - Loss:   906.5851 Validation Accuracy: 0.746094
Epoch  3, Batch 106 - Loss:   859.7889 Validation Accuracy: 0.746094
Epoch  3, Batch 107 - Loss:   844.2109 Validation Accuracy: 0.742188
Epoch  3, Batch 108 - Loss:   630.

KeyboardInterrupt: 